In [ ]:
#  Copyright 2017-2020 Reveal Energy Services, Inc
#
#  Licensed under the Apache License, Version 2.0 (the "License");
#  you may not use this file except in compliance with the License.
#  You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
#  limitations under the License.
#
# This file is part of Orchid and related technologies.
#

# Example: Using Pandas to Analyze Completion Parameters

This notebook illustrates using the Orchid* Python API and the pandas package to
perform completion analysis.

(*Orchid is a mark of Reveal Energy Services, Inc)

## 0.5 Import packages

The only import needed for the Python API is `orchid` itself.

In [ ]:
import orchid

The remaining imports are standard python packages to support the analysis.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# The following import is included for its "side-effects" of an improved color schemes and
# plot styles. (See the "Tip" in section 9.2 of "Python for Data Analysis" for details.)
import seaborn as sns
from scipy import integrate

## 1.0 Load the .ifrac project

The following code simply captures the configured location of the Orchid training data. It is not needed to
use the Orchid Python API itself, but it is used in this example to load well-known data.

In [ ]:
orchid_training_data_path = orchid.training_data_path()

In [ ]:
bakken_project = orchid.load_project(str(orchid_training_data_path.joinpath(
    'frankNstein_Bakken_UTM13_FEET.ifrac')))

## 2.0 Define a function to compute the stage treatment data

In [ ]:
def compute_stage_treatment_aggregates(treatment_stage):
    
    def slurry_rate_per_min_to_per_second_conversion_factor():
        source_slurry_rate_unit = treatment_curves['Slurry Rate'].sampled_quantity_unit()
        target_slurry_rate_unit = \
            f'{orchid.slurry_rate_volume_unit(source_slurry_rate_unit)}/s'
        local_result = orchid.get_conversion_factor(source_slurry_rate_unit,
                                                    target_slurry_rate_unit)
        return local_result

    def slurry_rate_bbl_per_second_to_gal_per_second_conversion_factor():
        local_result = orchid.get_conversion_factor('bbl/s', 'gal/s')
        return local_result

    treatment_curves = treatment_stage.treatment_curves()
    if not treatment_curves:
        return None
    
    raw_treatment_series = {n: c.time_series() for n, c in (treatment_curves.items())}
    stage_start_time = np.datetime64(stage.start_time)
    stage_stop_time = np.datetime64(stage.stop_time)
    
    fluid_per_sec = \
        (raw_treatment_series[orchid.SLURRY_RATE][stage_start_time:stage_stop_time] *
         slurry_rate_per_min_to_per_second_conversion_factor())
    stage_fluid = integrate.trapz(fluid_per_sec.values, 
                                  (fluid_per_sec.index - stage_start_time).seconds)
    
    stage_concentration = \
        raw_treatment_series[orchid.PROPPANT_CONCENTRATION][stage_start_time:stage_stop_time]
    proppant_per_sec = (fluid_per_sec * 
                        slurry_rate_bbl_per_second_to_gal_per_second_conversion_factor() *
                        stage_concentration)
    stage_proppant = integrate.trapz(proppant_per_sec.values, 
                                     (proppant_per_sec.index - stage_start_time).seconds)
    
    stage_pressure = \
        raw_treatment_series[orchid.TREATING_PRESSURE][stage_start_time:stage_stop_time]
    median_stage_pressure = stage_pressure.median()
    
    # Assumes that all three curves have the same time basis; that is, 
    # that the index for each series is equal.
    treatment_curves = pd.DataFrame.from_dict(raw_treatment_series)
    treatment_curves['dt'] = ((treatment_curves.index.values - stage_start_time) /
                              np.timedelta64(1, 's'))
    
    return stage_fluid, stage_proppant, median_stage_pressure

## 3.0 Build a pandas data frame

In [ ]:
# Remember the project units
bakken_units = {'length': bakken_project.unit_abbreviation('length'),
         'mass': bakken_project.unit_abbreviation('mass'),
         'pressure': bakken_project.unit_abbreviation('pressure'),
         'slurry volume': orchid.slurry_rate_volume_unit(bakken_project.unit_abbreviation('slurry rate')),
         'proppant mass': orchid.proppant_concentration_mass_unit(
             bakken_project.unit_abbreviation('proppant concentration'))}

In [ ]:
# Calculate the stage results
stage_results = []
for well in bakken_project.wells:
    stages = list(well.stages)

    for stage in stages:
        
        treatment_aggregates = compute_stage_treatment_aggregates(stage)
        # Skip stages with no aggregates. These stages most likely are from 
        # an untreated monitor well.
        if not treatment_aggregates:
            continue
            
        stage_fluid, stage_proppant, median_stage_pressure = treatment_aggregates
        stage_results.append((bakken_project.name, well.name, stage.display_stage_number,
                              stage.md_top(bakken_units['length']).magnitude,
                              stage.md_bottom(bakken_units['length']).magnitude,
                              stage_fluid, stage_proppant, median_stage_pressure))

In [ ]:
# Provide a way to manage DataFrame column names (which include units) using simpler,
# semantic identifiers. Creating the column names helps me avoid "typos" involved in
# getting the (project-specific) units correct, but is not necessary in a typical
# interactive session.

bakken_columns = {'project': 'Project',
                  'well': 'Well',
                  'stage': 'Stage',
                  'md_top': f'MD Top ({bakken_units["length"]})',
                  'md_bottom': f'MD Bottom ({bakken_units["length"]})',
                  'total_fluid': f'Total Fluid ({bakken_units["slurry volume"]})',
                  'total_proppant': f'Total Proppant ({bakken_units["proppant mass"]})',
                  'median_treating': f'Median Treating Pressure ({bakken_units["pressure"]})'}

In [ ]:
# Create the data frame
bakken_summaries = pd.DataFrame(data=stage_results,
                                columns=bakken_columns.values())
bakken_summaries.head()

### 3.1 Compute the stage length directly from the data frame

In [ ]:
bakken_summaries[f'Stage Length ({bakken_units["length"]})'] = \
    bakken_summaries.apply(
        lambda s: s[bakken_columns['md_bottom']] -
                  s[bakken_columns['md_top']], axis=1)
bakken_columns['stage_length'] = bakken_summaries.columns[-1]
bakken_summaries.head()

### 3.2 Now compute the proppant loading for each stage

In [ ]:
bakken_summaries[f'Proppant loading ({bakken_units["proppant mass"]}/{bakken_units["length"]})'] = \
    bakken_summaries.apply(
        lambda s: s[bakken_columns['total_proppant']] /
                  s[bakken_columns['stage_length']], axis=1)
bakken_columns['proppant_loading'] = bakken_summaries.columns[-1]
bakken_summaries.head()

## 4.0 Completion questions

### 4.1 What is the median proppant intensity per well?

In [ ]:
bakken_summaries[[bakken_columns['well'],
                  bakken_columns['proppant_loading']]]. \
    groupby(bakken_columns['well']).median()

In [ ]:
groups = bakken_summaries.groupby(bakken_columns['well'])

fig, ax = plt.subplots()
ax.margins(0.05) # Optional, just adds 5% padding to the autoscaling
for name, group in groups:
    ax.plot(group[bakken_columns['stage']],
            group[bakken_columns['proppant_loading']],
            marker='o', linestyle='', ms=6, label=name)
ax.legend()
plt.rcParams['figure.dpi'] = 150
plt.show()

That's a little hard to interpret with the outliers

In [ ]:
# Plot
groups = bakken_summaries[bakken_summaries[bakken_columns['stage']] > 5]. \
    groupby(bakken_columns['well'])

fig, ax = plt.subplots()
ax.margins(0.05) # Optional, just adds 5% padding to the autoscaling
for name, group in groups:
    ax.plot(group[bakken_columns['stage']],
            group[bakken_columns['proppant_loading']],
            marker='o', linestyle='', ms=6, label=name)
ax.legend()
ax.set_title(f'Proppant Loading by Stage')
ax.set_xlabel('Stage Number')
ax.set_ylabel(bakken_columns['proppant_loading'])
plt.rcParams['figure.dpi'] = 150
plt.show()

That's a little busy. Let's clean it up.

In [ ]:
def build_proppant_loading_plot(data_frame):
    groups = data_frame.groupby(bakken_columns['well'])

    fig, ax = plt.subplots(len(groups), sharex=True, sharey=True)
    fig.suptitle(f'{bakken_columns["proppant_loading"]} by Stage')
    i=0
    colors=['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple',
            'tab:brown', 'tab:pink', 'tab:gray', 'tab:olive', 'tab:cyan']
    for name, group in groups:
        ax[i].margins(0.05)
        ax[i].plot(group[bakken_columns['stage']],
                   group[bakken_columns['proppant_loading']],
                   marker='o', linestyle='', ms=6, label=name)
        ax[i].legend()
        ax[i].set_xlabel('Stage Number')
        i = i+1
        # Hide x labels and tick labels for all but bottom plot.
    for a in ax:
        a.label_outer()
    plt.rcParams['figure.dpi'] = 150
    plt.show()

build_proppant_loading_plot(bakken_summaries[bakken_summaries[bakken_columns['stage']]>5])

### 4.2 What is the median treating pressure per well?

In [ ]:
bakken_summaries[[bakken_columns['well'],
                  bakken_columns['median_treating']]]. \
    groupby(bakken_columns['well']).median()

In [ ]:
groups = bakken_summaries.groupby(bakken_columns['well'])

fig, ax = plt.subplots(3, sharex=True, sharey=True)
fig.suptitle(f'{bakken_columns["median_treating"]} by Stage')
i=0
colors=['tab:blue', 'tab:orange', 'tab:green',
        'tab:purple', 'tab:yellow', 'tab:red', 'tab:black']
for name, group in groups:
    ax[i].margins(0.05)
    ax[i].plot(group[bakken_columns['stage']],
               group[bakken_columns['median_treating']],
               colors[i], marker='o', linestyle='', ms=6, label=name)
    ax[i].legend()
    ax[i].set_xlabel('Stage Number')
    i = i+1

    # Hide x labels and tick labels for all but bottom plot.
for a in ax:
    a.label_outer()
plt.show()

## 5.0 Add another .ifrac project

In [ ]:
montney_project = orchid.load_project(str(orchid_training_data_path.joinpath(
    'Project-frankNstein_Montney_UTM13_METERS.ifrac')))

In [ ]:
# Remember the project units
montney_units = {'length': montney_project.unit_abbreviation('length'),
         'mass': montney_project.unit_abbreviation('mass'),
         'pressure': montney_project.unit_abbreviation('pressure'),
         'slurry volume': orchid.slurry_rate_volume_unit(
             montney_project.unit_abbreviation('slurry rate')),
         'proppant mass': orchid.proppant_concentration_mass_unit(
             montney_project.unit_abbreviation('proppant concentration'))}

In [ ]:
# Calculate the stage results for the Montney project
stage_results = []
for well in montney_project.wells:
    stages = list(well.stages)

    for stage in stages:

        treatment_aggregates = compute_stage_treatment_aggregates(stage)
        # Skip stages with no aggregates. These stages most likely are from
        # an untreated monitor well.
        if not treatment_aggregates:
            continue

        stage_fluid, stage_proppant, median_stage_pressure = treatment_aggregates
        stage_results.append((montney_project.name, well.name, stage.display_stage_number,
                              stage.md_top(montney_units['length']).magnitude,
                              stage.md_bottom(montney_units['length']).magnitude,
                              stage_fluid, stage_proppant, median_stage_pressure))

In [ ]:
# Provide a way to manage DataFrame column names (which include units) using simpler,
# semantic identifiers.

montney_columns = {'project': 'Project',
                  'well': 'Well',
                  'stage': 'Stage',
                  'md_top': f'MD Top ({montney_units["length"]})',
                  'md_bottom': f'MD Bottom ({montney_units["length"]})',
                  'total_fluid': f'Total Fluid ({montney_units["slurry volume"]})',
                  'total_proppant': f'Total Proppant ({montney_units["proppant mass"]})',
                  'median_treating': f'Median Treating Pressure ({montney_units["pressure"]})'}

In [ ]:
# Create the Montney data frame
montney_summaries = pd.DataFrame(data=stage_results,
                                columns=montney_columns.values())

montney_summaries[f'Stage Length ({montney_units["length"]})'] = \
    montney_summaries.apply(
        lambda s: s[montney_columns['md_bottom']] -
                  s[montney_columns['md_top']], axis=1)
montney_columns['stage_length'] = montney_summaries.columns[-1]

montney_summaries[f'Proppant loading ({montney_units["proppant mass"]}/'
                  f'{montney_units["length"]})'] = \
    montney_summaries.apply(
        lambda s: s[montney_columns['total_proppant']] /
                  s[montney_columns['stage_length']], axis=1)
montney_columns['proppant_loading'] = montney_summaries.columns[-1]

montney_summaries.head()

### 5.1 Combine the data for the two projects.

Because the frankNstein_Bakken_UTM13_FEET project is in US Oilfield units and the
Project-frankNstein_Montney_UTM13_METERS project is in metric units, I'll need to
convert one to the other.

In [ ]:
# Calculate the stage results in US oilfield units for the Montney project
stage_results = []
for well in montney_project.wells:
    stages = list(well.stages)

    for stage in stages:

        treatment_aggregates = compute_stage_treatment_aggregates(stage)
        if not treatment_aggregates:
            continue

        # Convert stage aggregates to US oilfield units
        stage_fluid = treatment_aggregates[0] * \
                      orchid.get_conversion_factor(montney_units['slurry volume'],
                                                   bakken_units['slurry volume'])
        stage_proppant = treatment_aggregates[1] *\
                         orchid.get_conversion_factor(montney_units['proppant mass'],
                                                      bakken_units['proppant mass'])
        median_stage_pressure = treatment_aggregates[2] * \
                                orchid.get_conversion_factor(montney_units['pressure'],
                                                             bakken_units['pressure'])

        stage_results.append((montney_project.name, well.name,
                              stage.display_stage_number,
                              stage.md_top(bakken_units['length']).magnitude,
                              stage.md_bottom(bakken_units['length']).magnitude,
                              stage_fluid, stage_proppant, median_stage_pressure))

In [ ]:
# Create the converted Montney data frame
montney_us_summaries = pd.DataFrame(data=stage_results,
                                    columns=list(bakken_columns.values())[:-2])

montney_us_summaries[f'Stage Length ({bakken_units["length"]})'] = \
    montney_us_summaries.apply(
        lambda s: s[bakken_columns['md_bottom']] -
                  s[bakken_columns['md_top']], axis=1)

montney_us_summaries[f'Proppant loading ({bakken_units["proppant mass"]}/'
                  f'{bakken_units["length"]})'] = \
    montney_us_summaries.apply(
        lambda s: s[bakken_columns['total_proppant']] /
                  s[bakken_columns['stage_length']], axis=1)

montney_us_summaries.head()

In [ ]:
combined = bakken_summaries[bakken_summaries[bakken_columns['stage']] > 5]\
    .append(montney_us_summaries)
combined.head()

In [ ]:
combined[[bakken_columns['well'], bakken_columns['proppant_loading']]]\
    .groupby(bakken_columns['well'])\
    .median()

In [ ]:
combined[[bakken_columns['well'], bakken_columns['proppant_loading']]]\
    .groupby(bakken_columns['well'])\
    .agg({bakken_columns['proppant_loading']:['median', 'std']})

In [ ]:
build_proppant_loading_plot(combined)